# Training a Char RNN to Tell Jokes

**Objective**: Build text generation model to be able to complete a sentence for different standup comedians


## Data Gathering

In [2]:
import requests               # a simple HTTP library for Python
from bs4 import BeautifulSoup # a great library for webscrapping
import pickle    

In [1]:
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="elementor-element elementor-element-74af9a5b elementor-widget elementor-widget-theme-post-content").find_all('p')]
    print(url)
    return text

# URLs of transcripts in scope
urls = ['http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/',
        'http://scrapsfromtheloft.com/2017/08/07/bo-burnham-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/']

# Comedian names
comedians = ['louis', 'dave', 'ricky', 'bo', 'bill', 'jim', 'john', 'hasan', 'ali', 'anthony', 'mike', 'joe']

In [3]:
# Requesting transcripts
transcripts = [url_to_transcript(u) for u in urls]

http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/
http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/
http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/
http://scrapsfromtheloft.com/2017/08/07/bo-burnham-2013-full-transcript/
http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/
http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/
http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/
http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/
http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/
http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/
http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/
http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-

In [4]:
!mkdir transcripts

for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)

In [5]:
# Load pickle file
data = {}
for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [6]:
# Take a look at the data
data.keys() ,data['louis'][:1]

(dict_keys(['louis', 'dave', 'ricky', 'bo', 'bill', 'jim', 'john', 'hasan', 'ali', 'anthony', 'mike', 'joe']),
 ['Intro\nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily agree with you, but I appreciate very much. Well, this is a nice place. This is easily the nicest place For many miles in every direction. That’s how you compliment a building And shit on a town with one sentence. It is odd around here, as I was driving here. There doesn’t seem to be any difference Between the sidewalk and the street for pedestrians here. People just kind of walk in the middle of the road. I love traveling And seeing all the different parts of the country. I live in New York. I live in a– There’s no value to your doing that at all.'])

## Data Cleaning

In [7]:
def combine_text(list_of_text):
    '''
    Take a list of texts and combine them into one large chunk of text
    
    Return a text (larger one)
    '''
    return ' '.join(list_of_text)
    

In [8]:
# Combining
data_combined = {key: [combine_text(value)] for key, value in data.items()}

In [9]:
import pandas as pd
pd.set_option('max_colwidth', 150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df.head()

,transcript
ali,"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have ..."
anthony,"Thank you. Thank you. Thank you, San Francisco. Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my s..."
bill,"[cheers and applause] All right, thank you! Thank you very much! Thank you. Thank you. Thank you. How are you? What’s going on? Thank you. It’s a ..."
bo,Bo What? Old MacDonald had a farm E I E I O And on that farm he had a pig E I E I O Here a snort There a Old MacDonald had a farm E I E I O [Appla...
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."


In [10]:
data_dave_unclean = data_df.iloc[4].values

In [11]:
len(data_dave_unclean[0])

48194

In [12]:
import re   
import string

def clean_text_re(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation,
       remove words containing numbers, additional punctuation and non-sensical text.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

step1 = lambda x: clean_text_re(x)

# Let clean the data:
data_clean = pd.DataFrame(data_df.transcript.apply(step1))
data_df.to_pickle("corpus.pkl")
print('DATA after cleaning with regular expression')
data_clean.head()

DATA after cleaning with regular expression


,transcript
ali,ladies and gentlemen please welcome to the stage ali wong hi hello welcome thank you thank you for coming hello hello we are gonna have to get thi...
anthony,thank you thank you thank you san francisco thank you so much so good to be here people were surprised when i told em i was gonna tape my special ...
bill,all right thank you thank you very much thank you thank you thank you how are you whats going on thank you its a pleasure to be here in the great...
bo,bo what old macdonald had a farm e i e i o and on that farm he had a pig e i e i o here a snort there a old macdonald had a farm e i e i o this i...
dave,this is dave he tells dirty jokes for a living that stare is where most of his hard work happens it signifies a profound train of thought the alch...


In [13]:
data_clean.to_csv('comedian_transcripts.csv')

In [14]:
ali_df = data_clean.iloc[0].values
anthony_df = data_clean.iloc[1].values
bill_df = data_clean.iloc[2].values
bo_df = data_clean.iloc[3].values
dave_df = data_clean.iloc[4].values
hasan_df = data_clean.iloc[5].values
jim_df = data_clean.iloc[6].values
joe_df = data_clean.iloc[7].values
john_df = data_clean.iloc[8].values
louis_df = data_clean.iloc[9].values
mike_df = data_clean.iloc[10].values
ricky_df = data_clean.iloc[11].values

In [15]:
ali_df_unclean = data_df.iloc[0].values
anthony_df_unclean = data_df.iloc[1].values
bill_df_unclean = data_df.iloc[2].values
bo_df_unclean = data_df.iloc[3].values
dave_df_unclean = data_df.iloc[4].values
hasan_df_unclean = data_df.iloc[5].values
jim_df_unclean = data_df.iloc[6].values
joe_df_unclean = data_df.iloc[7].values
john_df_unclean = data_df.iloc[8].values
louis_df_unclean = data_df.iloc[9].values
mike_df_unclean = data_df.iloc[10].values
ricky_df_unclean = data_df.iloc[11].values

In [17]:
len(hasan_df_unclean[0])

56094

In [51]:
combined_comedian = ali_df[0]+anthony_df[0]+bill_df[0]+bo_df+dave_df+hasan_df+jim_df+joe_df+john_df+louis_df+mike_df+ricky_df

In [ ]:
textfile_combined = open('combined_script.txt','w')
textfile_combined.write(combined_comedian[0])
textfile_combined.close()

## TextGenRnn

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip3 install git+git://github.com/minimaxir/textgenrnn.git

  Cloning git://github.com/minimaxir/textgenrnn.git to /tmp/pip-req-build-yrbv_m6c
  Running command git clone -q git://github.com/minimaxir/textgenrnn.git /tmp/pip-req-build-yrbv_m6c
     |████████████████████████████████| 454.3MB 40kB/s 
     |████████████████████████████████| 471kB 43.1MB/s 
     |████████████████████████████████| 6.0MB 38.5MB/s 
  Created wheel for textgenrnn: filename=textgenrnn-2.0.0-cp37-none-any.whl size=1734434 sha256=8b21060ee4abd98af7c1db5b722d954f40887654002041bb01af19e308708b74
  Stored in directory: /tmp/pip-ephem-wheel-cache-00pb8cpj/wheels/51/cd/43/32656d2da914e1bda73eeafa357602d92a9f67b587dfdd5aa8
Successfully built textgenrnn
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation:

In [12]:
from google.colab import files
import textgenrnn
from datetime import datetime
import os
import keras

Using TensorFlow backend.


In [18]:
from textgenrnn import textgenrnn

textgen = textgenrnn()

In [19]:
print('Input:', ali_df_unclean[0][102:141] )
print('Target:', ali_df_unclean[0][141])

Input: for coming. Hello! Hello. We are gonna 
Target: h


In [ ]:
textgen_ali = textgenrnn(name = 'aliwong_model')
textgen_ali.train_on_texts(ali_df_unclean,
                            rnn_bidirectional = True,
                            rnn_size = 128,
                            rnn_layers = 3,
                            max_length = 40,
                            dim_embeddings = 300,
                            num_epochs = 15,
                            gen_epochs = 15)

print(textgen_ali.model.summary())

Training on 38,969 character sequences.
Epoch 1/15
304/304 [==============================] - 39s 98ms/step - loss: 1.4777
Epoch 2/15
304/304 [==============================] - 31s 101ms/step - loss: 1.2804
Epoch 3/15
304/304 [==============================] - 31s 100ms/step - loss: 1.1670
Epoch 4/15
304/304 [==============================] - 31s 101ms/step - loss: 1.0882
Epoch 5/15
304/304 [==============================] - 30s 100ms/step - loss: 1.0177
Epoch 6/15
304/304 [==============================] - 31s 101ms/step - loss: 0.9525
Epoch 7/15
304/304 [==============================] - 31s 101ms/step - loss: 0.9020
Epoch 8/15
304/304 [==============================] - 31s 101ms/step - loss: 0.8413
Epoch 9/15
304/304 [==============================] - 31s 101ms/step - loss: 0.7817
Epoch 10/15
304/304 [==============================] - 31s 101ms/step - loss: 0.7242
Epoch 11/15
304/304 [==============================] - 31s 100ms/step - loss: 0.6687
Epoch 12/15
304/304 [==============

In [ ]:
textgen_ali.generate(1, prefix = 'My husband', max_gen_length=100)

My husband at home six-week Chinese every dad. And it was like, “But… I don’t wanna die!” You wanna



In [ ]:
textgen_Dave = textgenrnn(name = 'davechappelle_model')
textgen_Dave.train_on_texts(data_dave_unclean,
                            rnn_bidirectional = True,
                            rnn_layers = 3,
                            max_length = 40,
                            rnn_size = 128,
                            dim_embeddings = 300,
                            num_epochs = 20,
                            gen_epochs = 20)

print(textgen_Dave.model.summary())

Training new model w/ 3-layer, 128-cell Bidirectional LSTMs
Training on 48,195 character sequences.
Epoch 1/20
376/376 [==============================] - 56s 132ms/step - loss: 2.5085
Epoch 2/20
376/376 [==============================] - 49s 131ms/step - loss: 1.8703
Epoch 3/20
376/376 [==============================] - 50s 132ms/step - loss: 1.6569
Epoch 4/20
376/376 [==============================] - 49s 129ms/step - loss: 1.5097
Epoch 5/20
376/376 [==============================] - 48s 128ms/step - loss: 1.3949
Epoch 6/20
376/376 [==============================] - 48s 128ms/step - loss: 1.2943
Epoch 7/20
376/376 [==============================] - 48s 128ms/step - loss: 1.1977
Epoch 8/20
376/376 [==============================] - 48s 128ms/step - loss: 1.1014
Epoch 9/20
376/376 [==============================] - 48s 128ms/step - loss: 1.0026
Epoch 10/20
376/376 [==============================] - 48s 129ms/step - loss: 0.9033
Epoch 11/20
376/376 [==============================] - 48s 

In [ ]:
textgen_bill = textgenrnn(name='billburr_model')
textgen_bill.train_on_texts(bill_df_unclean,
                            rnn_bidirectional = True,
                            rnn_layers = 3,
                            max_length = 40,
                            rnn_size = 128,
                            dim_embeddings = 300,
                            num_epochs = 20,
                            gen_epochs = 20)

print(textgen_bill.model.summary())

Training new model w/ 3-layer, 128-cell Bidirectional LSTMs
Training on 64,608 character sequences.
Epoch 1/20
504/504 [==============================] - 88s 162ms/step - loss: 2.4993
Epoch 2/20
504/504 [==============================] - 81s 161ms/step - loss: 1.7908
Epoch 3/20
504/504 [==============================] - 82s 163ms/step - loss: 1.6191
Epoch 4/20
504/504 [==============================] - 83s 164ms/step - loss: 1.4974
Epoch 5/20
504/504 [==============================] - 82s 162ms/step - loss: 1.4005
Epoch 6/20
504/504 [==============================] - 85s 168ms/step - loss: 1.3161
Epoch 7/20
504/504 [==============================] - 84s 167ms/step - loss: 1.2321
Epoch 8/20
504/504 [==============================] - 83s 165ms/step - loss: 1.1520
Epoch 9/20
504/504 [==============================] - 83s 164ms/step - loss: 1.0682
Epoch 10/20
297/504 [================>.............] - ETA: 33s - loss: 0.9656

In [ ]:
textgen_hasan = textgenrnn(name = 'hasan_model')
textgen_hasan.train_on_texts(hasan_df_unclean,
                            rnn_bidirectional = True,
                            rnn_layers = 3,
                            max_length = 40,
                            rnn_size = 128,
                            dim_embeddings = 300,
                            num_epochs = 20,
                            gen_epochs = 20)

print(textgen_hasan.model.summary())

NotFoundError: ignored

In [ ]:
textgen_jim = textgenrnn(name = 'jim_model')
textgen_jim.train_on_texts(jim_df_unclean,
                            rnn_bidirectional = True,
                            rnn_layers = 3,
                            max_length = 40,
                            rnn_size = 128,
                            dim_embeddings = 300,
                            num_epochs = 20,
                            gen_epochs = 20)

print(textgen_jim.model.summary())

Training on 58,681 character sequences.
Epoch 1/20
458/458 [==============================] - 164s 351ms/step - loss: 1.7317
Epoch 2/20
458/458 [==============================] - 162s 353ms/step - loss: 1.3706
Epoch 3/20
458/458 [==============================] - 162s 353ms/step - loss: 1.2382
Epoch 4/20
458/458 [==============================] - 161s 352ms/step - loss: 1.1595
Epoch 5/20
458/458 [==============================] - 162s 353ms/step - loss: 1.0925
Epoch 6/20
458/458 [==============================] - 161s 351ms/step - loss: 1.0340
Epoch 7/20
458/458 [==============================] - 161s 352ms/step - loss: 0.9796
Epoch 8/20
458/458 [==============================] - 161s 353ms/step - loss: 0.9359
Epoch 9/20
458/458 [==============================] - 162s 353ms/step - loss: 0.8888
Epoch 10/20
458/458 [==============================] - 161s 351ms/step - loss: 0.8435
Epoch 11/20
458/458 [==============================] - 159s 347ms/step - loss: 0.8003
Epoch 12/20
458/458 [==

In [ ]:
textgen_joe = textgenrnn(name = 'joerogan_model')
textgen_joe.train_on_texts(joe_df_unclean,
                            rnn_bidirectional = True,
                            rnn_layers = 3,
                            max_length = 40,
                            rnn_size = 128,
                            dim_embeddings = 300,
                            num_epochs = 20,
                            gen_epochs = 20)

print(textgen_joe.model.summary())

NotFoundError: ignored

In [ ]:
textgen_john = textgenrnn(name = 'john_model')
textgen_john.train_on_texts(john_df_unclean,
                            rnn_bidirectional = True,
                            rnn_layers = 3,
                            max_length = 40,
                            rnn_size = 128,
                            dim_embeddings = 300,
                            num_epochs = 20,
                            gen_epochs = 20)

print(textgen_john.model.summary())

## Generating New Text

In [19]:
textgen_ali = textgenrnn('aliwong_model_weights.hdf5')

In [20]:
textgen_ali.generate(1,prefix='Today I', max_gen_length=150)

Today I was like, “You need to cleanse want to talk to you feel like you know? I don’t worry of hold on it– I’m just because you gotta shit the side 



In [21]:
textgen_hasan = textgenrnn('hasan_model_weights.hdf5')

In [22]:
textgen_hasan.generate(1, prefix = 'Today I', max_gen_length=150)

Today I realized the big shit really competne. I got this shit on your culture, but it’s not a letter than you got to go.” Then he says we pay to get



In [23]:
textgen_bill = textgenrnn('billburr_model_weights (1).hdf5')

In [24]:
textgen_bill.generate(1, prefix='Today I', max_gen_length=150)

Today I got to be home in the movie. You got to fly a helicopter, but you got to stay the bot loaded on the whole though, all I’m gonna do today. I’m



In [25]:
textgen_dave = textgenrnn('davechappelle_model_weights (1).hdf5')

In [27]:
textgen_dave.generate(1, prefix = 'Today I', max_gen_length=150)

Today Iimiam. Here that stufe, and the wedch from every ghetto ♪ On Marco, but the fuck out that would be like it.” And I got to take what I said, “B



In [28]:
textgen_jim = textgenrnn('jim_model_weights.hdf5')

In [30]:
textgen_jim.generate(1, prefix = 'Today I', max_gen_length=150)

Today I’m like, “Hey, why does my guy only happen in my actor and I just don’t know Jim Jefferies are you, “Well…” And I said, “All right. I go up to



In [31]:
textgen_joe = textgenrnn('joerogan_model_weights.hdf5')

In [32]:
textgen_joe.generate(1, prefix = 'Today I', max_gen_length=150)

Today I’ve never been a lot of things that whole job, ’cause this white people in some respect! We get a problem with a problem with this? Is the Whi



In [33]:
textgen_john = textgenrnn('john_model_weights.hdf5')

In [34]:
textgen_john.generate(1, prefix = 'Today I', max_gen_length=150)

Today I get to mytinnving a river  years all the parents were like, “Thanks, it’s like, “Hey, I’m allegana. Hey! – Suremor Nichols were like… “Ah, th



In [35]:
def joke_telling_robot(comedian, prefix, max_gen_length):
  if comedian == 'Ali':
    text = textgen_ali.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  if comedian == 'Dave':
    text = textgen_dave.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  if comedian == 'Bill':
    text = textgen_bill.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  if comedian == 'Hasan':
    text = textgen_hasan.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  if comedian == 'Jim':
    text = textgen_jim.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  if comedian == 'Joe':
    text = textgen_joe.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  if comedian == 'John':
    text = textgen_john.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  

In [66]:
def all_comedians(prefix, max_gen_length):
  print('Ali Wong:') 
  textgen_ali.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  print('Dave Chappelle:')
  textgen_dave.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  print('Bill Burr:')
  textgen_bill.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  print('Hasan Minhaj:')
  textgen_hasan.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  print('Jim Jeffries:')
  textgen_jim.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  print('Joe Rogan:') 
  textgen_joe.generate(1, prefix = prefix, max_gen_length=max_gen_length)
  print('John Mulaney:')
  textgen_john.generate(1, prefix = prefix, max_gen_length=max_gen_length)

In [67]:
all_comedians('American politics are', 100)

Ali Wong:
American politics are your last year on our word. I just wanna leave his eyes and I am, have some u

Dave Chappelle:
American politics are that was the fuck up to a black dude that was coming out of that shit. They w

Bill Burr:
American politics are bums arsies. “You scare you, they fight it’s like a buly little offensive or 

Hasan Minhaj:
American politics area. I didn’t know what I do. “Hasan Minhaj. You’ll need you!” “You’re all thing

Jim Jeffries:
American politics are allowed to see a lot of strip who see everyone else whole very here, you can’

Joe Rogan:
American politics are. That’s why the fuck when you watch you’re not getting sucg? ‘Cause he has ev

John Mulaney:
American politics are smoochiaaially should serve how three within she said, “Ah-he’d be real as a 



In [69]:
all_comedians('Kids today are', 100)

Ali Wong:
Kids today are the look at the time, I do a quick shit in the baby. I don’t know what’s a big dad a

Dave Chappelle:
Kids today are your whole stars. I’d still watch a little bide. And then I wanted to have a high th

Bill Burr:
Kids today are ruined the street, they ask the shooter with the planes, out you this fucking religi

Hasan Minhaj:
Kids today are saying. I’m like, “Hasan bhainteen it’s back and if each other is secredied in this 

Jim Jeffries:
Kids today are fucking keep with my guns. I deserve to earn as model. Now… See, me and the first th

Joe Rogan:
Kids today are men can don’t. Just get those shills just starts sleeping on?” I have a six-year-pla

John Mulaney:
Kids today are magtive estaien tcievilly. And we were like, “Well, I did a cubicle. And it would st



In [74]:
all_comedians('The other day I', 100)

Ali Wong:
The other day I have the knicket. I meant had a nutramsial and it all fucked. That’s– the time in t

Dave Chappelle:
The other day I didn’t remember lady-hass you can imagine. I know him, when shit. I hold ten I didn

Bill Burr:
The other day I have the fucking leftoring about the movie. You hear to do that make sex sandwich. 

Hasan Minhaj:
The other day I has ever seen this shit right now. I’m like, “I know the way it is as a great. You 

Jim Jeffries:
The other day I can look at the locance that works out of you wanna pass.” And then they see each o

Joe Rogan:
The other day I can’t gotta really step. This is the world we get to make a kid trapperbul shit whe

John Mulaney:
The other day I’ get he really     you know who that would never walk in, it’s packer when and he w



In [36]:
joke_telling_robot('Ali', 'My Husband is', 200)

My Husband is that this huge but if it ans as me from her staychare. That’s how all welcomethed and you gotta change it anything. So. I just wanna be a number in front of my company for a floo and I 



In [37]:
joke_telling_robot('Bill', 'My life is', 200)

My life is fucking leftore. Dude, he’s gonna get there when I was alrieved as your fored in 20 years, you go to pick your funning and all the sake of my wife. I’m like, “Where I don’t give a low in t



In [38]:
joke_telling_robot('Hasan', 'Politics are', 200)

Politics are around. I was like, “You know what kid US? You’re not an Indian girllib. “Well, you know whut you think you go to push. I’ll never make a school angrant the first little br0wn parents ar



In [39]:
joke_telling_robot('Dave', 'People today are', 200)

People today are paides to anything every ghetto ♪ ♪ Hollywood with her fuck up. They’re gonna go to stood up the fuck up. She doesn’t know the work to watch a white around the steps one than and had



In [43]:
joke_telling_robot('Joe', 'In this world', 200)

In this world we just starts really find that the calder. Every deas one of while the fuck are you ain’t the car, lase than anybody. The fuck are you ain’t the problem. We’re a girl in this fucking c



In [42]:
joke_telling_robot('John', 'My grandmother', 200)

My grandmothers are marren try hour show that it? I really like having about her .chaned to self, that a nursery.” Why buy the cow all famlin before my wife was saying Mary. I am as a lone that way t

